In [2]:
from typing import List, Tuple
import torch
import copy
import os
import sys

# custom
from AudioCaps.AudioCaps_Dataset import * # 데이터셋
from transformers import GPT2Tokenizer
from ClipCap_forAAC.CLIPCAP_forAAC import * # network
from Train import *
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

data_dir = './AudioCaps'

# PANNs를 써먹기 위해 prefix_size를 수정
audio_prefix_size = 15
semantic_prefix_size = 11 
prefix_size = audio_prefix_size + semantic_prefix_size

transformer_num_layers = {"audio_num_layers" : 4, "semantic_num_layers" : 4}
prefix_size_dict = {"audio_prefix_size" : audio_prefix_size, "semantic_prefix_size" : semantic_prefix_size}


# vocab_size===============
# Custom Tokeniezr1 : 7911
# Custom Tokeniezr2 : 5084
# ACT Tokeniezr : 5069
# Clotho Tokeniezr : 4992
# GPT2 Tokenizer : None
# vocab_size===============

vocab_size = None
tokenizer_type = None

if vocab_size ! = None :
    tokenizer = tokenizer_AudioCaps(vocab_size)
    tokenizer_type = 'Custom'
    model_param_name = 'model_AudioCaps_Custom_' + str(vocab_size) + ".pt"
else :
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    tokenizer_type = 'GPT2'
    model_param_name = 'model_AudioCaps_GPT2_header_freezing.pt'

TEST_BATCH_SIZE = 5
test_dataloader  = dataloader_AudioCapsDataset(tokenizer, data_dir, TEST_BATCH_SIZE, split = 'test', prefix_size = prefix_size, is_TrainDataset = False, tokenizer_type = tokenizer_type)

torch.cuda.empty_cache()

USE_CUDA = torch.cuda.is_available() 
device = torch.device('cuda:0' if USE_CUDA else 'cpu')

model = get_ClipCap_AAC(tokenizer, 
                        vocab_size = vocab_size, Dataset = 'AudioCaps',
                        prefix_size_dict = prefix_size_dict, transformer_num_layers = transformer_num_layers, 
                        encoder_freeze = False, decoder_freeze = True,
                        pretrain_fromAudioCaps = False, device = device)

get dataset...: 100%|███████████████████████| 960/960 [00:00<00:00, 1403.23it/s]
/home/cuai5th/anaconda3/envs/pytorch1.8_p38/lib/python3.8/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=512 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/home/cuai5th/anaconda3/envs/pytorch1.8_p38/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


audio feature's mapping network : num_head = 8 num_layers = 4
semantic feature ver1's mapping network : num_head = 8 num_layers = 4
GPT2 freezing


In [4]:
model_path = './Train_record/' + model_param_name
model.load_state_dict(torch.load(model_path))

NameError: name 'model' is not defined

In [ ]:
eval_model(model, test_dataloader, 31, 'test', True)